In [1]:
# You should not modify code in this cell
import sys
import nltk
from nltk.corpus import treebank

# Get numsents POS-tagged sentences from the treebank corpus
def get_pos_data(numsents):

    # Extract required number of sentences
    sentences = treebank.tagged_sents()[:numsents]

    # Initialize
    sequences = []
    symbols = set()
    tag_set = set()
    
    # Go over each extracted sentence ...
    for sentence in sentences:
        for i in range(len(sentence)):
            word, tag = sentence[i]
            word = word.lower()  # normalize case
            symbols.add(word)    # add this word
            tag_set.add(tag)
            sentence[i] = (word, tag)  # store tagged token
        sequences.append(sentence)

    # Return sequences, the list of tags and all the words that we saw
    return sequences, list(tag_set), list(symbols)

# Train the transition and emission probabilities
def train():
    print('Training HMM...')

    # Use the first 5000 sentences from treebank corpus
    labelled_sequences, states, symbols = get_pos_data(5000)
    
    # Define the estimator to be used for probability computation
    estimator = lambda fd, bins: nltk.LidstoneProbDist(fd, 0.1, bins)
    
    # count occurences of starting states, transitions out of each state
    # and output symbols observed in each state
    freq_starts = nltk.FreqDist()
    freq_transitions = nltk.ConditionalFreqDist()
    freq_emissions = nltk.ConditionalFreqDist()
    for sequence in labelled_sequences:
        lasts = None
        for token in sequence:
            state = token[1]
            symbol = token[0]
            if lasts == None:
                freq_starts[state] += 1
            else:
                freq_transitions[lasts][state] += 1
            freq_emissions[state][symbol] += 1
            lasts = state

            # update the state and symbol lists
            if state not in states:
                states.append(state)
            if symbol not in symbols:
                symbols.append(symbol)

    # create probability distributions (with smoothing)
    N = len(states)
    starts = estimator(freq_starts, N)
    transitions = nltk.ConditionalProbDist(freq_transitions, estimator, N)
    emissions = nltk.ConditionalProbDist(freq_emissions, estimator, len(symbols))
                               
    print('Training Complete') 
    
    # Return the transition and emissions probabilities along with 
    # the list of all the states and output symbols
    return starts, transitions, emissions, states, symbols

In [4]:
from numpy import zeros, array, float32, int16, argmax
from math import log, exp
from nltk import re
import numpy

# call the train function (it will take some time)
priors, transitions, emissions, states, symbols = train()
# suggestion: inspect these five variables and the code in train() 
# to get a sense of the data and data structures

# write your viterbi code here
def viterbi(sentences):
    sent = re.compile(r"[\w']+|[(){}.,!?;]")
    observations = sent.findall(sentences)
    
    N = len(states)
    T = len(observations)

    matrix = numpy.zeros(shape=(N, T), dtype=float32)
    backpointer = {}
    
    for x in range(0,N):
        matrix[x][0] = priors.logprob(states[x]) + emissions[states[x]].logprob(observations[0])
        backpointer[x,0] = 0
        
    for y in range(1,T):
        for x in range(0,N):
            prev_state = None
            for z in range(0,N):
                current_prob = matrix[z][y-1]
                + transitions[states[z]].logprob(states[x])
                + emissions[states[x]].logprob(observations[y])
                if not prev_state or current_prob > prev_state[0]:
                    prev_state = (current_prob, states[z])
            matrix[x][y] = prev_state[0]
            backpointer[states[x],y] = prev_state[1]
    
    temp = None
    for n in range(0,N):
        prob = matrix[n,T-1]
        if not temp or prob > temp[0]:
            temp = (prob, states[n])
    
    curr_val = temp[1]
    best_path = [curr_val]
    for i in range (T-1, 0, -1):
        last_val = backpointer[curr_val,i]
        best_path.append(last_val)
        current_val = last_val
    
    best_path.reverse()
    print(best_path)
    return observations, best_path
                           

sentence = 'hello world' 
viterbi(sentence)


Training HMM...
Training Complete
['``', '-NONE-']


(['hello', 'world'], ['``', '-NONE-'])

In [7]:
from numpy import zeros, array, float32, int16, argmax
from math import log, exp
from nltk import re
# call the train function (it will take some time)
priors, transitions, emissions, states, symbols = train()
# suggestion: inspect these five variables and the code in train() 
# to get a sense of the data and data structures


# write your viterbi code here  
#pass in observed sentences 

def viterbi(string_words):
    sentence = re.compile(r"[\w']+|[(){}.,!?;]")
    observations = sentence.findall(string_words)
    
    N = len(states)
    T = len(observations)
    backpointer = {}
    matrix = numpy.zeros(shape=(N,T), dtype=float32)
    
    for x in range (0,N):
        matrix[x][0] = priors.logprob(states[x])
        + emissions[states[x]].logprob(observations[0])
        
        backpointer[x,0] = 0
        
    for y in range (1,T):
        for x in range (0,N):
            prev_state = None
            for z in range (0,N):
                current_prob = matrix[z][y-1]
                + transitions[states[z]].logprob(states[x])
                + emissions[states[x]].logprob(observations[y])
                if not prev_state or current_prob > prev_state[0]:
                    prev_state = (current_prob, states[z])
            matrix[x][y] = prev_state[0]
            backpointer[states[x],y] = prev_state[1]
            
    temp = None
    for n in range(0,N):
        current_val = matrix[n, T-1]
        if not temp or current_val > temp[0]:
            temp = (current_val, states[n])
    current = temp[1]
    bestpath = [current]
    for i in range(T-1, 0, -1):
        last_val = backpointer[current,i]
        bestpath.append(last_val)
        current = last_val
    bestpath.reverse()
    print(bestpath)
    return observations, bestpath
    
    
    
s = 'hello world'
viterbi(s)

Training HMM...
Training Complete
['DT', '-NONE-']


(['hello', 'world'], ['DT', '-NONE-'])

In [ ]:
# open test-sentences
testSentenceFile = open('test-sentences.txt')
tagViterbi(testSentenceFile) # you need to define a tagViterbi function

In [ ]:
# eventually, open the hw-sentences
hwSentenceFile = open('hw-sentences.txt')
tagViterbi(hwSentenceFile) 